## Treino — Surrogate robusto
Este bloco treina um surrogate XGBoost (MultiOutput) usando o dataset novo gerado com `rocket_dataset_generator.py` + `airrocket/simulate.py`.
Ele prevê **apogeu** e métricas de **estabilidade robusta** (mínimo ao longo do voo e em maxQ), além de envelope (Mach e aceleração).

In [2]:
import os
import pandas as pd

from airrocket.surrogate import default_surrogate_spec, fit_surrogate, save_bundle

# === Parâmetros (ajuste aqui) ===
DATASET_PATH = "dataset/rocket_design_dataset5000.csv"  # gere esse CSV com o gerador novo
OUT_MODEL_PATH = "models/airrocket_surrogate.pkl"
FILTER_OK_ONLY = True

spec = default_surrogate_spec()

df = pd.read_csv(DATASET_PATH)
if FILTER_OK_ONLY and "ok" in df.columns:
    df = df[df["ok"] == True].copy()

# Mantém somente linhas com targets finitos
for t in spec.target_cols:
    df = df[pd.to_numeric(df[t], errors="coerce").notna()]

print("Rows:", len(df))
print("Targets:", spec.target_cols)

model = fit_surrogate(df, spec, random_state=42)
os.makedirs(os.path.dirname(OUT_MODEL_PATH), exist_ok=True)
save_bundle(OUT_MODEL_PATH, model=model, spec=spec)
print("Saved:", OUT_MODEL_PATH)

Rows: 2614
Targets: ('apogee', 'stability_min_over_flight', 'stability_at_max_q', 'max_mach', 'max_acceleration')
Saved: models/airrocket_surrogate.pkl
